In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/ssdata/clinvar/clinvar/parseClinvar.csv")

In [3]:
df = df.assign(idx=df.index)

In [4]:
df

,GeneID,variant,NM,label,idx
0,57539,Ala864Thr,NM_020779.4,True,0
1,112817,Gly287Val,NM_138413.4,True,1
2,112817,Arg70Pro,NM_138413.4,True,2
3,112817,Cys257Gly,NM_138413.4,True,3
4,22995,Gln265Pro,NM_001194998.2,True,4
...,...,...,...,...,...
70820,23152,Pro2506Gln,NM_001386298.1,False,70820
70821,2906,Thr1082Ala,NM_000836.4,False,70821
70822,4810,Ser1646Phe,NM_001291867.2,False,70822
70823,23133,Met291Thr,NM_015107.3,False,70823


In [65]:
sf = pd.read_csv("/ssdata/clinvar/clinvar/variant_summary.txt",delimiter="\t")

/home/dz/anaconda3/envs/bio/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
sf = sf.assign(nmid=sf.Name.apply(lambda s: s[:s.find("(")]))

In [67]:
sf = sf.assign(variant=sf.Name.apply(lambda s: s[s.find("(p.") + 3 : -1]))

In [68]:
ff = pd.merge(df,sf,how="inner",left_on=["NM","variant"], right_on=["nmid","variant"]).drop_duplicates(subset=["idx"])

In [70]:
aaTable = {"Ala":"A",
           "Arg": "R",
           "Asn": "N",
           "Asp": "D",
           "Cys": "C",
           "Gln": "Q",
           "Glu": "E",
           "Gly": "G",
           "His": "H",
           "Ile": "I",
           "Leu": "L",
           "Lys": "K",
           "Met": "M",
           "Phe": "F",
           "Pro": "P",
           "Ser": "S",
           "Thr": "T",
           "Trp": "W",
           "Tyr": "Y",
           "Val": "V"}

In [11]:
from processBioDBs.utilities import getSequence

In [12]:
import pickle
seqs = pickle.load(open("/ssdata/clinvar/clinvar/seqs.pkl","rb"))

In [13]:
symbols = set(ff.GeneSymbol)

symbols = [s for s in symbols if s not in seqs]

In [17]:
skippedSymbols = set()

In [18]:
from tqdm.notebook import tqdm
for s in tqdm(symbols):
    if s not in seqs:
        try:
            seqs[s] = getSequence(s)
        except:
            skippedSymbols.add(s)

  0%|          | 0/500 [00:00<?, ?it/s]

In [71]:
import numpy as np

In [72]:
skippedVariants = set()

variants = []

for idx,row in tqdm(ff.iterrows(),total=ff.shape[0]):

    o,L,v = aaTable[row.variant[:3]], int(row.variant[3:-3]) - 1, aaTable[row.variant[-3:]]
    if row.GeneSymbol not in seqs or L >= len(seqs[row.GeneSymbol]) or seqs[row.GeneSymbol][L] != o:
        skippedVariants.add(idx)
        variants.append(np.nan)
    else:
        variant = s[:L] + v + s[L+1:]
        variants.append(variant)

  0%|          | 0/70825 [00:00<?, ?it/s]

In [73]:
1 - len(skippedVariants)/ff.shape[0]

0.9627109071655489

In [74]:
ff = ff.assign(variantSeq=variants)

In [75]:
ff

,GeneID_x,variant,NM,label,idx,#AlleleID,Type,Name,GeneID_y,GeneSymbol,...,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,nmid,variantSeq
0,57539,Ala864Thr,NM_020779.4,True,0,15062,single nucleotide variant,NM_020779.4(WDR35):c.2590G>A (p.Ala864Thr),57539,WDR35,...,-,N,"ClinGen:CA339782,UniProtKB:Q9P2L0#VAR_064582,O...",3,23,20133230,C,T,NM_020779.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
2,112817,Gly287Val,NM_138413.4,True,1,15069,single nucleotide variant,NM_138413.4(HOGA1):c.860G>T (p.Gly287Val),112817,HOGA1,...,-,N,"ClinGen:CA113813,UniProtKB:Q86XE5#VAR_064036,O...",3,30,99371292,G,T,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
4,112817,Arg70Pro,NM_138413.4,True,2,15072,single nucleotide variant,NM_138413.4(HOGA1):c.209G>C (p.Arg70Pro),112817,HOGA1,...,-,N,"ClinGen:CA113816,OMIM:613597.0005",3,33,99344669,G,C,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
6,112817,Cys257Gly,NM_138413.4,True,3,15073,single nucleotide variant,NM_138413.4(HOGA1):c.769T>G (p.Cys257Gly),112817,HOGA1,...,-,N,"ClinGen:CA113818,UniProtKB:Q86XE5#VAR_064035,O...",3,34,99361682,T,G,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
8,22995,Gln265Pro,NM_001194998.2,True,4,15094,single nucleotide variant,NM_001194998.2(CEP152):c.794A>C (p.Gln265Pro),22995,CEP152,...,-,N,"ClinGen:CA210936,UniProtKB:O94986#VAR_063813,O...",3,55,49085556,T,G,NM_001194998.2,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143892,23152,Pro2506Gln,NM_001386298.1,False,70820,1166322,single nucleotide variant,NM_001386298.1(CIC):c.7517C>A (p.Pro2506Gln),23152,CIC,...,-,N,-,2,1176081,42799306,C,A,NM_001386298.1,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
143894,2906,Thr1082Ala,NM_000836.4,False,70821,1166329,single nucleotide variant,NM_000836.4(GRIN2D):c.3244A>G (p.Thr1082Ala),2906,GRIN2D,...,-,N,-,2,1176712,48946427,A,G,NM_000836.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
143896,4810,Ser1646Phe,NM_001291867.2,False,70822,1166440,single nucleotide variant,NM_001291867.2(NHS):c.4937C>T (p.Ser1646Phe),4810,NHS,...,-,N,-,2,1175807,17750565,C,T,NM_001291867.2,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
143898,23133,Met291Thr,NM_015107.3,False,70823,1166476,single nucleotide variant,NM_015107.3(PHF8):c.872T>C (p.Met291Thr),23133,PHF8,...,-,N,-,2,1175826,54037629,A,G,NM_015107.3,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...


In [76]:
ff.to_csv("/ssdata/clinvar/clinvar/parseClinvarWithVariant.csv")